# 01 - Creation du dataset des données brutes de metrics fractionnées

Ce notebook génère :

- 1 fichier csv "raw_merge_metrics_dataset.csv" qui fusionne les colonnes fractionnées avec le dataset d'origine
- 1 fichier json "metrics_events_dict.json" pour lister les code d'identification des évènements

**Etapes de création :**

- Pour chaque colonne contenant des valeurs de type list ou dict

    - Fractionnement des colonnes en dataframe

    - Fusion des dataframes issus de la ou des fraction(s)

- Fusion des colonnes fractionnées avec les colonnes non fratcionnées du dataset de départ

## Imports

In [1]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path

## 1. Création dataset metrics

In [2]:
# source path to raw metrics dataset
filename = 'metrics.csv'
path = '../data/raw/'
source_csv = os.path.join(path, filename)
# target path to save metrics dictionnaire
save_json ='../data/metrics/metrics_events_dict.json'
# target path to save merge raw metrics dataset
save_csv = '../data/metrics/raw_merge_metrics_dataset.csv'

### a) Import des données brutes

In [3]:
# téléchargement dans le repertoire 'data' d'un fichiers 'csv' depuis le blob Azure
from azure_blob import download_blob_file
download_blob_file(file_name=filename, local_path=path)

metrics.csv successfully downloaded!


In [4]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(filepath_or_buffer=Path(source_csv)).sort_values(by='created_at')
metrics_df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
metrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164430 entries, 0 to 1164429
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   id                         1164430 non-null  int64  
 1   status                     1164430 non-null  object 
 2   created_at                 1164430 non-null  object 
 3   cyan_capacity              0 non-null        float64
 4   cyan_remaining             0 non-null        float64
 5   magenta_capacity           0 non-null        float64
 6   magenta_remaining          0 non-null        float64
 7   yellow_capacity            0 non-null        float64
 8   yellow_remaining           0 non-null        float64
 9   black_capacity             0 non-null        float64
 10  black_remaining            0 non-null        float64
 11  machineId                  1164430 non-null  int64  
 12  connected_operators        1164430 non-null  object 
 13  varnishLevel

In [5]:
# suppression des colonnes ne contenant aucune valeurs
metrics_df = metrics_df.dropna(axis=1)
# suppression de la colonne machineId
metrics_df = metrics_df.drop('machineId', axis=1)
# visualisation des 3 premières lignes
metrics_df.head(3)

,id,status,created_at,connected_operators,varnishLevelsTargetvolume,varnishLevelsTotalvolume,modules,events
0,4169748,WARNING,2022-04-15 05:55:06.678000,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",36192.322612,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]
1,4169749,WARNING,2022-04-15 05:55:06.829000,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",36192.322612,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...","[{""source"": ""PLC"", ""message"": "" JV-Ti non prêt..."
2,4169753,WARNING,2022-04-15 05:55:14.494000,"[{""name"": ""Viktor"", ""level"": ""Operator""}]",36192.322612,100000,"[{""sn"": """", ""name"": ""Print Engine 1"", ""type"": ...",[]


### b) Fractionnement des colonnes contenant des listes

In [6]:
# on verifie le type des valeurs contenu dans les colonnes de type objet contenant des listes
print('modules :', type(metrics_df.modules.loc[0]))
print('events :', type(metrics_df.events.loc[0]))
print('connected_operators :', type(metrics_df.connected_operators.loc[0]))

modules : <class 'str'>
events : <class 'str'>
connected_operators : <class 'str'>


In [7]:
# fonction retournant le dataframe d'une colonne fractionnée
# col=colonne à fractionner
# df=dataframe source
# data=dict des colonnes du df à conserver dans le df à retourner
def convert_col_to_df(col, df, data=None):
    
    # création du dictionnaire de données vide
    if data == None :
        data = {}
    # ou liste des clés du dictionnaire input
    else :
        data_keys = list(data.keys())

    # on converti le type des valeurs str en list
    if not isinstance(df[col].loc[0], list):
        df[col] = df[col].apply(lambda x : json.loads(x))

    # liste des clés du dictionnaire de la colonne à partir de la première occurence
    # on recherche la première occurence non vide et de type list 
    # pour l'affecter à une variable first
    for i in range(0, (len(df[col]))):
        value = df[col].loc[i]
        if len(value) > 0 and isinstance(value, list):
            first = value[0]
            print('first : ', type(first), first)
            break

    # on liste les clés du dictionnaire de l'occurence
    col_keys = first.keys()
    for ck in col_keys :
        data[ck+'_'+col] = []

    # on itére dans la serie pour récupérer les valeurs et les stocker dans le dictionnaire data
    for i in range(df.index.start, df.index.stop):
        # evaluation des valeurs 'str' en 'list'
        values = df[col].loc[i]
        if isinstance(values, list) and len(values) > 0 :
            # ajout des valeurs dans le dictionnaire 'd'
            for value in values :
                for k in value.keys():
                    data[k+'_'+col].append(value.get(k))
                for dk in data_keys:
                    data[dk].append(df[dk].loc[i])

    # re-assignation de la variable df
    df = pd.DataFrame(data)

    return df

In [8]:
# id temoin pour vérifier le fractionnement et la fusion des colonnes
check_id = 4170152

In [9]:
# détail de la ligne témoin
check_line = metrics_df[metrics_df.id == check_id]
print('######## id %d ######## ' %check_id)
print(check_line.values)
print('######## id %d modules details ######## ' %check_id)
print(json.loads(check_line.modules.values[0])[0].get('counters'))
print(json.loads(check_line.modules.values[0])[1].get('counters'))
print('######## id %d events details ######## ' %check_id)
print(json.loads(check_line.events.values[0])[0])
print(json.loads(check_line.events.values[0])[1])

######## id 4170152 ######## 
[[4170152 'IDLE' '2022-04-15 06:06:35.404000'
  '[{"name": "Viktor", "level": "Operator"}]' 36192.322612308 100000
  '[{"sn": "", "name": "Print Engine 1", "type": "Varnish Printer", "counters": [{"name": "3D Varnish Counter", "value": 1792992}], "generation": ""}, {"sn": "", "name": "iFoil L", "type": "iFoil", "counters": [{"name": "Total Pages Counter", "value": 22881}, {"name": "Foiled Pages Counter", "value": 31092}], "generation": "Gen. 2"}]'
  '[{"source": "iFoil", "message": " JV-Ti non prêt : impression impossible", "timestamp": "2022-04-15T06:06:56.278Z", "criticality": "INFO", "identification": "391"}, {"source": "PLC", "message": " En attente", "timestamp": "2022-04-15T06:06:56.418Z", "criticality": "INFO", "identification": "330"}]']]
######## id 4170152 modules details ######## 
[{'name': '3D Varnish Counter', 'value': 1792992}]
[{'name': 'Total Pages Counter', 'value': 22881}, {'name': 'Foiled Pages Counter', 'value': 31092}]
######## id 4170

#### 1) Colonne 'connected_operators'

In [10]:
# creation d'un dataframe connected_operators (~42s)
connected_operators_df = convert_col_to_df('connected_operators', metrics_df, {'id':[]})
print(connected_operators_df.info())
connected_operators_df.head(2)

first :  <class 'dict'> {'name': 'Viktor', 'level': 'Operator'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164430 entries, 0 to 1164429
Data columns (total 3 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   id                         1164430 non-null  int64 
 1   name_connected_operators   1164430 non-null  object
 2   level_connected_operators  1164430 non-null  object
dtypes: int64(1), object(2)
memory usage: 26.7+ MB
None


,id,name_connected_operators,level_connected_operators
0,4169748,Viktor,Operator
1,4169749,Viktor,Operator


#### 2) Colonne 'events'

In [11]:
# creation d'un dataframe events (~20s)
events_df = convert_col_to_df('events', metrics_df, {'id':[]})
print(events_df.info())
events_df.head(2)

first :  <class 'dict'> {'source': 'PLC', 'message': ' JV-Ti non prêt : impression impossible', 'timestamp': '2022-04-15T05:55:23.462Z', 'criticality': 'INFO', 'identification': '391'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81361 entries, 0 to 81360
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     81361 non-null  int64 
 1   source_events          81361 non-null  object
 2   message_events         81361 non-null  object
 3   timestamp_events       81361 non-null  object
 4   criticality_events     81361 non-null  object
 5   identification_events  81361 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.7+ MB
None


,id,source_events,message_events,timestamp_events,criticality_events,identification_events
0,4169749,PLC,JV-Ti non prêt : impression impossible,2022-04-15T05:55:23.462Z,INFO,391
1,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391


##### Identification

In [12]:
# liste des codes d'identification
identification_codes_list = events_df['identification_events'].unique()
np.sort(identification_codes_list)

array(['0', '2', '311', '313', '315', '320', '321', '322', '323', '324',
       '325', '326', '327', '329', '330', '331', '332', '333', '334',
       '343', '344', '345', '346', '349', '350', '351', '352', '354',
       '355', '356', '357', '358', '359', '371', '372', '376', '377',
       '380', '381', '384', '385', '386', '387', '388', '389', '391',
       '405', '406', '407', '408', '411', '416', '417', '418', '419',
       '430', '440', '444', '445', '446', '447', '453', '454', '460',
       '466', '471', '472', '475', '476', '479', '480',
       'ICB communication error', 'Kernel_Error',
       'Pilot communication error', 'RCB communication error',
       'iFoil communication error'], dtype=object)

In [13]:
# liste des évènements uniques
identification_dict = {}
c = 1
id_list = []
for i in range(events_df.index.start, events_df.index.stop):
    id = events_df.identification_events.loc[i]
    if id not in id_list:
        id_list.append(id)
        identification_dict[id] = events_df.message_events.loc[i]
        c += 1
identification_dict

{'391': ' JV-Ti non prêt : impression impossible',
 '330': ' En attente',
 '332': ' Disponible',
 '377': ' Chargeur: mode auto non activé',
 '333': ' Plateau de têtes en mouvement',
 '334': ' Préchauffage',
 '331': ' Impression en cours',
 'Kernel_Error': 'Cannot find enough search zones on the sheet (42,77%), please check your margins\n',
 '315': ' Support lingette non installé',
 '417': ' Portes margeur ouvertes E-0417',
 '406': ' Purge en cours',
 '407': ' Essuyage en cours',
 '352': ' Double feuille détectée E-0352',
 '344': ' Manque papier',
 'ICB communication error': 'A communication error has occured between RCB n° 1 and  ICB n°5',
 '376': ' Réception: mode auto non activé',
 '445': " Erreur plaque d'empreintes E-0445",
 '325': ' Erreur de com. tapis impression E-0325',
 '343': ' Nettoyage tapis en cours',
 '345': " Consommable à 80% d'usure",
 '358': ' Démarrage machine',
 '453': ' Erreur capteur lift palette C2 E-0453',
 '381': ' Capot après têtes ouvert E-0381',
 '354': ' Bo

##### Source

In [14]:
# liste des sources
source_list = events_df['source_events'].unique()
np.sort(source_list)

array(['ICB n°1', 'ICB n°2', 'ICB n°4', 'ICB n°5', 'ICB n°6', 'ICB n°7',
       'ICB n°8', 'Kernel', 'PLC', 'Pilot', 'RCB n°1', 'RCB n°2',
       'RCB n°3', 'iFoil'], dtype=object)

##### Criticality

In [15]:
# liste des sources
criticality_list = events_df['criticality_events'].unique()
np.sort(criticality_list)

array(['ERROR', 'INFO', 'WARNING'], dtype=object)

##### Events Json dict

In [16]:
# on sauvegarde la liste des codes d'identification d'event
with open(file=Path(save_json), mode="r+", encoding='utf-8') as jsonFile:
    try :
        # chargement des données du fichier dans un dictionnaire
        data = json.load(jsonFile)
        # ajout des données dans le dictionnaire
        data['identification'] = identification_dict
        data['criticality'] = list(np.sort(criticality_list)),
        data['source'] = list(np.sort(source_list))
        # définit la position actuelle du fichier à l'offset
        jsonFile.seek(0)
        # écriture du dicitonnaire dans le fichier
        json.dump(data, jsonFile, indent=4, ensure_ascii=False)
    except ValueError as e:
        print(e)
    finally :
        # fermeture du fichier
        jsonFile.close()

#### 3) Colonne 'modules'

In [17]:
# creation d'un dataframe modules (~1m15s)
modules_df = convert_col_to_df('modules', metrics_df, {'id':[]})
# visualisation des données
print(modules_df.info())
modules_df.head(2)

first :  <class 'dict'> {'sn': '', 'name': 'Print Engine 1', 'type': 'Varnish Printer', 'counters': [{'name': '3D Varnish Counter', 'value': 1792992}], 'generation': ''}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2328860 entries, 0 to 2328859
Data columns (total 6 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   id                  int64 
 1   sn_modules          object
 2   name_modules        object
 3   type_modules        object
 4   counters_modules    object
 5   generation_modules  object
dtypes: int64(1), object(5)
memory usage: 106.6+ MB
None


,id,sn_modules,name_modules,type_modules,counters_modules,generation_modules
0,4169748,,Print Engine 1,Varnish Printer,"[{'name': '3D Varnish Counter', 'value': 17929...",
1,4169748,,iFoil L,iFoil,"[{'name': 'Total Pages Counter', 'value': 2288...",Gen. 2


##### Colonne counters

In [18]:
# creation d'un dataframe counters (~2m7s)
counters_df = convert_col_to_df('counters_modules', modules_df, {'type_modules':[], 'id': []})
# visualisation des données
print(counters_df.info())
counters_df.head(2)

first :  <class 'dict'> {'name': '3D Varnish Counter', 'value': 1792992}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3493290 entries, 0 to 3493289
Data columns (total 4 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   type_modules            object
 1   id                      int64 
 2   name_counters_modules   object
 3   value_counters_modules  int64 
dtypes: int64(2), object(2)
memory usage: 106.6+ MB
None


,type_modules,id,name_counters_modules,value_counters_modules
0,Varnish Printer,4169748,3D Varnish Counter,1792992
1,iFoil,4169748,Total Pages Counter,22881


In [19]:
# verification de l'intégrité des données
counters_check_line = counters_df[counters_df.id == check_id]
counters_check_line

,type_modules,id,name_counters_modules,value_counters_modules
210,Varnish Printer,4170152,3D Varnish Counter,1792992
211,iFoil,4170152,Total Pages Counter,22881
212,iFoil,4170152,Foiled Pages Counter,31092


### c) Fusion des dataframes des colonnes fractionnées

#### 1) Merge modules et counters

In [20]:
# fusion du df modue et du df counter
merge_modules_df = pd.merge(modules_df, counters_df, on=['id','type_modules'])
# suppression de la colonne fractionnées
merge_modules_df = merge_modules_df.drop(['counters_modules'], axis=1)
# verification de l'intégrité des données
module_check_line = merge_modules_df[merge_modules_df.id == check_id]
module_check_line

,id,sn_modules,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules
210,4170152,,Print Engine 1,Varnish Printer,,3D Varnish Counter,1792992
211,4170152,,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881
212,4170152,,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092


In [21]:
merge_modules_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3493290 entries, 0 to 3493289
Data columns (total 7 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   id                      int64 
 1   sn_modules              object
 2   name_modules            object
 3   type_modules            object
 4   generation_modules      object
 5   name_counters_modules   object
 6   value_counters_modules  int64 
dtypes: int64(2), object(5)
memory usage: 213.2+ MB


#### 2) Merge operators

In [22]:
# on merge avec le df operators en fonction de l'id de message
merge_operators_df = pd.merge(merge_modules_df, connected_operators_df, on='id', suffixes=['','_op'])
# verification de l'intégrité des données
op_check_line = merge_operators_df[merge_operators_df.id == check_id]
op_check_line

,id,sn_modules,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators
210,4170152,,Print Engine 1,Varnish Printer,,3D Varnish Counter,1792992,Viktor,Operator
211,4170152,,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator
212,4170152,,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator


#### 3) Merge events

In [23]:
# on crée un df, à partir du df events, ne contenant que les lignes ayant un évènement de source Ifoil
events_ifoil = events_df[events_df.source_events == 'iFoil']
# on crée un df, à partir du df merge, ne contenant que les lignes ayant un module de type Ifoil
module_ifoil = merge_operators_df[merge_operators_df.type_modules == 'iFoil']
# on merge les deux df des lignes Ifoil en focntion de l'id de message
merge_ifoil_df = pd.merge(events_ifoil, module_ifoil, how='outer', on='id', suffixes=['_event','_module'])
# verification de l'intégrité des données
events_check_line = merge_ifoil_df[merge_ifoil_df.id == check_id]
events_check_line

,id,source_events,message_events,timestamp_events,criticality_events,identification_events,sn_modules,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators
0,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391,,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator
1,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391,,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator


Comme il y avait 2 counters ('Total Pages Counter' et 'Foiled Pages Counter') pour le module de type 'Ifoil', nous avons bien 2 lignes.

In [24]:
# on crée un df, à partir du df events, ne contenant que les lignes ayant un évènement de source Ifoil
events_no_ifoil = events_df[events_df.source_events != 'iFoil']
# on crée un df, à partir du df merge, ne contenant que les lignes ayant un module de type Ifoil
module_no_ifoil = merge_operators_df[merge_operators_df.type_modules != 'iFoil']
# on merge les deux df des lignes Ifoil en focntion de l'id de message
merge_no_ifoil_df = pd.merge(events_no_ifoil, module_no_ifoil, how='outer', on='id', suffixes=['_event','_module'])
# verification de l'intégrité des données
events_check_line = merge_no_ifoil_df[merge_no_ifoil_df.id == check_id]
events_check_line

,id,source_events,message_events,timestamp_events,criticality_events,identification_events,sn_modules,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators
1,4170152,PLC,En attente,2022-04-15T06:06:56.418Z,INFO,330,,Print Engine 1,Varnish Printer,,3D Varnish Counter,1792992,Viktor,Operator


##### Concaténation

In [25]:
# on concatene les df ifoi et no_ifoil pour ne perdre aucune valeur
concat_events_df = pd.concat([merge_ifoil_df, merge_no_ifoil_df])
# verification de l'intégrité des données
events_check_line = concat_events_df[concat_events_df.id == check_id]
events_check_line

,id,source_events,message_events,timestamp_events,criticality_events,identification_events,sn_modules,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators
0,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391,,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator
1,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391,,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator
1,4170152,PLC,En attente,2022-04-15T06:06:56.418Z,INFO,330,,Print Engine 1,Varnish Printer,,3D Varnish Counter,1792992,Viktor,Operator


In [26]:
concat_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510431 entries, 0 to 1175124
Data columns (total 14 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   id                         int64 
 1   source_events              object
 2   message_events             object
 3   timestamp_events           object
 4   criticality_events         object
 5   identification_events      object
 6   sn_modules                 object
 7   name_modules               object
 8   type_modules               object
 9   generation_modules         object
 10  name_counters_modules      object
 11  value_counters_modules     int64 
 12  name_connected_operators   object
 13  level_connected_operators  object
dtypes: int64(2), object(12)
memory usage: 401.7+ MB


#### 4) Merge metrics

In [27]:
# dernier merge de toutes les colonnes
merge_metrics_df = pd.merge(concat_events_df, metrics_df, how='outer', on='id', suffixes=['','_metrics'])
# suppression des colonnes fractionnées
merge_metrics_df = merge_metrics_df.drop(['connected_operators','modules','events'], axis=1)
# verification de l'intégrité des données
metrics_check_line = merge_metrics_df[merge_metrics_df.id == check_id]
metrics_check_line

,id,source_events,message_events,timestamp_events,criticality_events,identification_events,sn_modules,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,created_at,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391,,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,IDLE,2022-04-15 06:06:35.404000,36192.322612,100000
1,4170152,iFoil,JV-Ti non prêt : impression impossible,2022-04-15T06:06:56.278Z,INFO,391,,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,IDLE,2022-04-15 06:06:35.404000,36192.322612,100000
2,4170152,PLC,En attente,2022-04-15T06:06:56.418Z,INFO,330,,Print Engine 1,Varnish Printer,,3D Varnish Counter,1792992,Viktor,Operator,IDLE,2022-04-15 06:06:35.404000,36192.322612,100000


In [28]:
merge_metrics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510431 entries, 0 to 3510430
Data columns (total 18 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   id                         int64  
 1   source_events              object 
 2   message_events             object 
 3   timestamp_events           object 
 4   criticality_events         object 
 5   identification_events      object 
 6   sn_modules                 object 
 7   name_modules               object 
 8   type_modules               object 
 9   generation_modules         object 
 10  name_counters_modules      object 
 11  value_counters_modules     int64  
 12  name_connected_operators   object 
 13  level_connected_operators  object 
 14  status                     object 
 15  created_at                 object 
 16  varnishLevelsTargetvolume  float64
 17  varnishLevelsTotalvolume   int64  
dtypes: float64(1), int64(3), object(14)
memory usage: 508.9+ MB


## 2. Outout csv

In [29]:
merge_metrics_df.to_csv(path_or_buf=Path(save_csv))